In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-08_4fon5
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-08_4fon5
     |████████████████████████████████| 64 kB 2.3 MB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369221 sha256=cdb058abb5c8f5120d094afc92a36d8a8e48ca931f8ed21daf0e6c3db4c653e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-pijneu2t/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=e2a87eb6dcf62234b09e737b76ca62744723d21bd9aed2519cb77c9622624d56
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built clip ftfy


### Libraries

In [3]:
import numpy as np
import torch
import nnabla as nn
import nnabla.parametric_functions as PF

import clip

2022-02-01 15:59:33,868 [nnabla][INFO]: Initializing CPU extension...


### Select Model

In [4]:
model, _ = clip.load('ViT-B/32') # 'ViT-B/16', 'ViT-L/16'
model = model.cpu()

100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 194MiB/s]


### output file name

In [5]:
out_file = './ViT-B-32.h5'

### utils

In [6]:
def pytorch_to_nn_param_map():
    '''map from tensor name to Nnabla default parameter names
    '''
    return {
        'weight': 'W',
        'bias': 'b',
        '.': '/'
    }

def rename_params(param_name):
    pytorch_to_nn_dict = pytorch_to_nn_param_map()
    for k in pytorch_to_nn_dict:
        if k in param_name:
            param_name = param_name.replace(k, pytorch_to_nn_dict[k])
    return param_name

### Conversion

In [7]:
for k, v in model.named_parameters():
  key = rename_params(k)
  params = PF.get_parameter_or_create(key, shape=v.shape)
  params.d = v.detach().numpy()

### Save outputs

In [8]:
nn.parameter.save_parameters(out_file)

2022-02-01 15:59:45,721 [nnabla][INFO]: Parameter save (.h5): ./ViT-B-32.h5
